## Data

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import numpy as np
import pandas as pd
from pandasql import sqldf
import plotly.graph_objects as go
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [ ]:
file_name = 'processed-raw_data.csv'
df = pd.read_csv(f'../data/processed/{file_name}')

df['at'] = pd.to_datetime(df['at'])

## Analysis

### Games statistics

**1. What are the pot size distributions?**

In [ ]:
df['hand_id'].nunique()

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(10, 6))
pot_sizes = df.groupby('hand_id')['pot_size'].max().sort_values(ascending=False) / 0.20


pot_sizes.to_frame().hist(ax=axes[0], ec='white', bins=25)
axes[0].set_title('Raw pot size distribution')
axes[0].set_ylabel('Number of hands')
axes[0].set_xlabel('Big blinds')

np.log10(pot_sizes).to_frame().hist(ax=axes[1], ec='white')
axes[1].set_title('log(pot size) distribution')
axes[1].set_ylabel('Number of hands')
axes[1].set_xlabel('log(Big blinds)')

fig.suptitle('Median pot size is about 10^1.5 ~ 31 x BB')

The left chart (raw pot sizes) shows that the distribution is heavily skewed. It's easier to use log of pot size (right chart) and see that the median pot value is around 30x BB (~10**1.5)

**2. At which phases are the hands won?**

Judging from the result, we can conclude that the game was pretty casual as it often went to River. My hypothesis is that good players have better evaluation of their hands strength, hence they can make decisions earlier (eg. folding earlier). Meanwhile, at a lower stake game, player don't lose much in terms of dollar value hence it's ok to commit longer.

We can verify this by comparing hand end phase vs. a higher stake game.

In [ ]:
fig, ax = plt.subplots()
ax.clear()
df.loc[df['action']=='Won'].groupby('hand_id')['phase'].max().value_counts(dropna=False, ascending=True).plot(ax=ax, kind='barh', width=1, ec='white')
ax.set_title('At which phases are hands won?')
ax.set_ylabel('Phase')
ax.set_xlabel('Number of hands')

**3. How many pots were split?**

We can determine by counting the number of winners per hand

In [ ]:
df.loc[df['action']=='Won']['hand_id'].value_counts().value_counts()

### Individual player statistics

**1. Overview:  Number of hands played, Winnings, Win per hand**

By itself number of hands played only shows how many hands the players sit in. But will be useful to calculate the various playstyle metrics below.

We can make a reasonable assumption that, given enough hands, the better players win more. Next we will analyse different patterns of behaviors. Hopefully we can learn from the winners here.

In [ ]:
# pl = player dataframe to store all metrics
pl = df.loc[(df['phase']=='Pre-flop') & (df['rank-P0']==1)]['player_name'].value_counts(dropna=False).to_frame(name='num_hands_played')
_expenses = df.groupby('player_name')['put_in'].sum()
_expenses.name = 'total_expenses'


_income = df.loc[df['action']=='Won'].groupby('player_name')['_amount'].sum().sort_values()
_income.name = 'total_income'

pl = pl.merge(_income, left_index=True, right_index=True)
pl = pl.merge(_expenses, left_index=True, right_index=True)

pl['win/hand'] = (pl['total_income'] - pl['total_expenses']) / pl['num_hands_played']

pl.sort_values(by='win/hand', ascending=False).head(5)

In [ ]:
fig2, axes = plt.subplots(1, 3, figsize=(12, 3.5), sharey=True)
pl = pl.sort_values(by='num_hands_played')
pl15 = pl[-15:]
pl15['num_hands_played'].plot(ax=axes[0], title='Num hands played', kind='barh', width=1, ec='white')
(pl15['total_income'] - pl15['total_expenses']).plot(ax=axes[1], title='Net win (no. of BB)', kind='barh', ec='white', width=1)
pl15['win/hand'].plot(ax=axes[2], title='Win per hand (no of. BB)', kind='barh', ec='white', width=1)
axes[0].set_ylabel('player id')

In [ ]:
# bucket players into 'classes' based on BBs win per hand
# s = winning
# f = not so winning

class_s = pl.loc[pl['win/hand']>=.1].index
class_a = pl.loc[(pl['win/hand']>0) & (pl['win/hand']<.1)].index
class_b = pl.loc[(pl['win/hand']<=0) & (pl['win/hand']>-.1)].index
class_f = pl.loc[pl['win/hand']<=-.1].index

From this chart we can see that Loi @ 6k.. is a good player. We will try to find out how he plays!

**2. VPIP: Voluntarily Put In Pot & PFR: Pre-flop Raises**

VPIP tracks the percentage of hands in which a particular player voluntarily puts money into the pot preflop. VPIP increases when a player could fold but instead commits money to the pot preflop. This includes limping (merely calling the big blind), calling, and raising.

PFR tracks the percentage of hands in which a particular player makes a preflop raise when having the opportunity to fold or call instead. This includes reraises. By definition, PFR is a subset of VPIP. So we will plot the ratio PFR/VPIP to measure player's tendency.


In [ ]:
# tracking player win
# per hand
# cost = sum(put_in)
# income = sum(_amount) where action = won
# index = hand_num, hand_id
# columns = player_name
# value = income - cost
# 
# prob doesn't make sense to do this analysis because players play at different tables outside of this game
# a summary metric per session makes more sense

In [ ]:
# running vpip
# np.array([1,0,0,1,1,..])
# where
#   1 = vpip
#   0 = no vpip
# purpose: track if this figure change when the game progress or stack sizes change
# hypothesis:
#   players that are up will tighten their game. vice versa, those are down will play looser

# recap
# vpip is defined when phase == 'Pre-flop' and rank-P2 == 1 and action in ['Bet/Raises', 'Calls']

# player_tracker = {
#   player_name: pd.DataFrame with columns: hand_id, session_id, running vpip, pnl
# }

player_data = {}
for player_name in df['player_name'].unique():
    _df = df[(df['player_name']==player_name) & (df['phase']=='Pre-flop') & (df['rank-P2']==1)][['hand_num', 'hand_id', 'action']].copy(deep=True).reset_index(drop=True)
    
    _df['vpip'] = 0
    _df.loc[_df['action'].isin(['Bets/Raises', 'Calls']), 'vpip'] = 1
    
    _df['hand_num'] = _df['hand_num'].astype(int)
    _df['session_date'] = _df['hand_id'].str[:10]
    _df['running_vpip_all_time'] =_df['vpip'].cumsum()
    _df['running_vpip_session'] = _df.groupby('session_date')['vpip'].cumsum()
    
    
    _df['running_vpip_session_%'] = _df['running_vpip_session'] / _df['hand_num']
    _df['running_vpip_all_time_%'] = _df['running_vpip_all_time'] / _df.index

    player_data[player_name] = _df

In [ ]:
# fig, ax = plt.subplots(figsize=(15,5))
# x_data = sorted(df['session_date'].unique())
# ax.scatter(x_data, [0]*len(x_data), s=0)
# plt.xticks(rotation=90)

# # pdf = player df
# for pid, pdf in player_data.items():
#     # only plot frequent players to marke chart readable
#     if pdf['session_date'].nunique() > 3:
#         session_vpip = pdf.groupby('session_date')['vpip'].mean()
#         ax.plot(session_vpip, label=pid, markersize=30, lw=1)
#     # print(pid, session_vpip)

# ax.legend()

In [ ]:
# VPIP
_vpip = df.loc[(df['phase']=='Pre-flop') &
    (df['rank-P2']==1) &
    (df['action'].isin(['Bets/Raises', 'Calls']))
    ]['player_name'].value_counts(dropna=False)
_vpip.name = 'VPIP'
pl = pl.merge(_vpip, how='left', left_index=True, right_index=True)
pl['% VPIP'] = (pl['VPIP'] / pl['num_hands_played']).round(2)

# PFR / VPIP
_pfr = df.loc[(df['phase']=='Pre-flop') &
    (df['action']=='Bets/Raises')][['player_name', 'hand_num']].drop_duplicates()['player_name'].value_counts()
_pfr.name = 'PFR'
pl = pl.merge(_pfr, how='left', left_index=True, right_index=True)
pl['PFR / VPIP'] = (pl['PFR'] / pl['VPIP']).round(2)

In [ ]:
fig, ax = plt.subplots()
pl.iloc[-15:][['% VPIP', 'win/hand']].plot(kind='scatter', x='% VPIP', y='win/hand', ax=ax, s=30, label='frequent players')
pl.iloc[:-15][['% VPIP', 'win/hand']].plot(kind='scatter', x='% VPIP', y='win/hand', ax=ax, s=30, color='#8b8b8b', alpha=.5, label='infrequent players')


linear_regressor = LinearRegression()
linear_regressor.fit(pl.iloc[-15:]['% VPIP'].values.reshape(-1, 1), pl.iloc[-15:]['win/hand'].values.reshape(-1, 1))

X = np.linspace(0, 1, 1000).reshape(-1, 1)
y_pred = linear_regressor.predict(X)

ax.plot(X, y_pred, ls='--', lw=1)
ax.set_ylabel('win/hand (no. of BB)')

ax.legend(loc='center left', bbox_to_anchor=(1, .5),)

slope = round(linear_regressor.coef_[0][0],2)
ax.set_title('Higher % VPIP (playing more loosely)\nis correlated with lower win/hand')

In [ ]:
fig, ax = plt.subplots()
pl.iloc[-15:][['PFR / VPIP', 'win/hand']].plot(kind='scatter', x='PFR / VPIP', y='win/hand', ax=ax, s=30, label='frequent players')
pl.iloc[:-15][['PFR / VPIP', 'win/hand']].plot(kind='scatter', x='PFR / VPIP', y='win/hand', ax=ax, s=30, color='#8b8b8b', alpha=.5, label='infrequent players')


linear_regressor = LinearRegression()
linear_regressor.fit(pl.iloc[-15:]['PFR / VPIP'].values.reshape(-1, 1), pl.iloc[-15:]['win/hand'].values.reshape(-1, 1))

X = np.linspace(0, 1, 1000).reshape(-1, 1)
y_pred = linear_regressor.predict(X)

ax.plot(X, y_pred, ls='--', lw=1)
ax.set_ylabel('win/hand (no. of BB)')

ax.legend(loc='center left', bbox_to_anchor=(1, .5),)

slope = round(linear_regressor.coef_[0][0],2)
ax.set_title('Higher Pre-flop Raises (playing more aggressively)\nis correlated to higher win/hand')

It's clearer now that the winning players play fewer hands (lower % VPIP). But when they play, they are more aggressive with raisings!

**3. 3-BET and C-BET** (WIP)

In [ ]:
# _3bet = df.loc[(df['phase']=='Pre-flop') &
#     (df['action']=='Bets/Raises') &
#     (df['rank-A2']>1)]['player_name'].value_counts(dropna=False).fillna(0)
# _3bet.name = '3-BET'
# pl = pl.merge(_3bet, how='left', left_index=True, right_index=True)
# pl['% 3-BET'] = (pl['3-BET'] / pl['num_hands_played']).round(3)

In [ ]:
# pl.sort_values(by='% 3-BET', na_position='first')['% 3-BET'].plot(kind='barh', title='3-BET', width=1, ec='white')

**4. Wins and positions**

Good players play positions as well!
* EP: Early positions
* MP: Middle positions
* LP: Late positions

In [ ]:
_pos = df.loc[df['action']=='Won'].pivot_table(
    index='player_name',
    columns='position_tag',
    values='hand_num',
    aggfunc=len,
    fill_value=0,
)
_pos['Total'] = _pos.sum(axis=1)

_pos.columns.name = None
for col in _pos.columns:
    if col != 'Total':
        _pos[f'Win at {col}'] = _pos[col] / _pos['Total']

_pos = _pos.round(3)
_pos_pct = _pos[[col for col in _pos.columns if col.startswith('Win at')]]

_pos_pct

In [ ]:
# fig4, axes = plt.subplots(4, 1)
# _pos_pct.loc[class_s].plot(ax=axes[0], kind='barh', stacked=True, title='Share of winnings based on Position', width=1, ec='white')
# _pos_pct.loc[class_a].plot(ax=axes[1], kind='barh', stacked=True, width=1, ec='white')
# _pos_pct.loc[class_b].plot(ax=axes[2], kind='barh', stacked=True, width=1, ec='white')
# _pos_pct.loc[class_f].plot(ax=axes[3], kind='barh', stacked=True, width=1, ec='white')

# for ax in axes:
#     ax.get_yaxis().set_ticks([])

# axes[3].legend(loc='lower center', ncol=4, bbox_to_anchor=(0.5, -0.25))
# ax.set_ylabel('')